In [1]:
%matplotlib inline
import networkx as nx
import pandas as pd
import numpy as np

# Selecting data

In [2]:
def select_data(organization=['CDU', 'CSU', 'FDP', 'SPD', 'Left', 'Greens'], debate_ind=None): # 0 for 2010-05-07, 1 for 2012-02-27, 2 for 2015-02-27, 3 for 2015-07-17, 4 for 2015-08-19
    data = pd.read_pickle("../datasets/contest_data.pkl")
    
    # data = data[data['organization'].isin(organization)] # Selecting party
    # data = data[data['type'] == 'Speech'] # Selecting speeches
    # data = data[data['type'] != 'Speech'] # Selecting EoVs
    
    if debate_ind is not None:
        debate_dates = sorted(list(set(data['time'])))
        data = data[data['time'].isin([debate_dates[debate_ind]])]
    
    return data

In [3]:
data = select_data(organization=['CDU', 'CSU', 'FDP', 'SPD', 'Left', 'Greens'])
data = data[['concept', 'agreement', 'time']]

print(data)

                                      concept  agreement                 time
0                  Aid_Currency stabilisation          1  2010-05-07 12:00:00
1                            Crisis_Contagion          1  2010-05-07 12:00:00
2                                Alternatives          0  2010-05-07 12:00:00
4                     EMU_Self-responsibility          1  2010-05-07 12:00:00
5                         Solidarity_European          0  2010-05-07 12:00:00
...                                       ...        ...                  ...
6082          Crisis solution_Competitiveness          1  2015-08-19 12:00:00
6083       Crisis causes_Greece_Mismanagement          1  2015-08-19 12:00:00
6084  Crisis causes_Living beyond their means          1  2015-08-19 12:00:00
6085                              Aid_Success          1  2015-08-19 12:00:00
6087                               Aid_Growth          1  2015-08-19 12:00:00

[4694 rows x 3 columns]


# Computing contestedness score

In [4]:
def prepare_agreement_disagreement_data(data):
    
    concepts = {}
    
    for index, row in data.iterrows(): 
        if not row['concept'] in concepts:
            concepts[row['concept']] = [0, 0]
        concepts[row['concept']][row['agreement']] += 1
        
    return concepts

def get_contested_concepts(concept_dict, min_disagree=1, min_agree=1):
    concept_contested_dict = {}
    
    for k, v in concept_dict.items():
        if v[0] >= min_disagree and v[1] >= min_agree:
            concept_contested_dict[k] = v
            
    return concept_contested_dict

def get_max_agree_disagree(data_dict):
    data = pd.DataFrame(data_dict).T
    return np.max(data[0].to_numpy() + data[1].to_numpy()), np.max([data[0].to_numpy(), data[1].to_numpy()])

def compute_concept_index(data_dict, beta=0.5, max_values=None):
    
    if max_values is None:
        max_values, min_max_values = get_max_agree_disagree(data_dict)
        print(max_values, min_max_values)
    
    for k, values in data_dict.items():
        freq = sum(values) / max_values
        closeness = 1 - abs(values[0] - values[1]) / sum(values)
        max_v = max(values)
        min_v = min(values)
        score = freq * closeness * (min_v / min_max_values)
        score = np.power(score, 1/3)
        
        data_dict[k].append(score)
        
    data_list = list(data_dict.items())
    data_list = sorted(data_list, key=lambda x: x[-1][-1], reverse=True)
        
    return data_list

def show_concept_index(data_list, k=np.inf):
    
    for i,v in enumerate(data_list):
        print(v)
        if i+1 == k:
            break
        
cdata_dict = prepare_agreement_disagreement_data(data)
cdata_dict = get_contested_concepts(cdata_dict)
print(cdata_dict)
agree_disagree_all = get_max_agree_disagree(cdata_dict)
print(agree_disagree_all)
cdata_list = compute_concept_index(cdata_dict)
show_concept_index(cdata_list)

{'Aid_Currency stabilisation': [3, 24], 'Alternatives': [11, 23], 'Solidarity_European': [5, 32], 'Solidarity_Mutual': [1, 12], 'EU_SGP_Sanctions': [1, 18], 'Crisis causes_Living beyond their means': [3, 11], 'Responsibility_Greek EMU entry_Red-green': [1, 13], 'Crisis causes_Speculation': [4, 24], "Aid_EU's future": [1, 24], 'Aid_Lawfulness': [8, 2], 'Greece_Debt restructuring': [5, 17], 'Aid_Private sector involvement': [1, 29], 'Aid_Private creditors': [37, 4], 'Crisis causes_Greece_Mismanagement': [2, 20], 'Aid_Guaranty': [1, 39], 'Aid_Risk': [3, 23], 'Greece_Wage cuts': [23, 7], 'Greece_Political will': [76, 36], 'EU_Transfer union': [21, 1], 'EU_Unity': [7, 9], 'Financial regulation_Tax': [5, 18], 'Aid_Time gain': [1, 21], 'Party politics_FDP': [11, 1], 'Germany_Tax cuts': [3, 1], 'Unity_Government': [7, 1], 'Consensus seeking_Government': [3, 12], 'Crisis management_Aggravation_Government': [2, 15], 'Bundestag_Procedure': [7, 1], 'Crisis management_Persistence_Government': [17, 

In [5]:
# Computing average score
np.mean([cdata_list[i][1][2] for i in range(len(cdata_list))])

0.14765123766649485

# Splitting data into agree/disagree

In [6]:
cdata_dict_ad = prepare_agreement_disagreement_data(data)
cdata_dict_ad = get_contested_concepts(cdata_dict_ad, min_disagree=0, min_agree=0)

def split_concepts(concept_contested_dict):
    """
    This function splits the data into two dictionaries containing only agreement and disagreement data separately
    """
    agree_dict = {}
    disagree_dict ={}
    
    for k, val in concept_contested_dict.items():
        if val[0] > 0:
            disagree_dict[f"{k}_disagree"] = val[0]
        if val[1] > 0:
            agree_dict[f"{k}_agree"] = val[1]
            
    return agree_dict, disagree_dict

cdata_agree_dict, cdata_disagree_dict = split_concepts(cdata_dict_ad)
print(cdata_agree_dict)
print(cdata_disagree_dict)

{'Aid_Currency stabilisation_agree': 24, 'Crisis_Contagion_agree': 25, 'Alternatives_agree': 23, 'EMU_Self-responsibility_agree': 14, 'Solidarity_European_agree': 32, 'Aid_Self-help_agree': 13, 'Solidarity_Mutual_agree': 12, 'EU_SGP_Sanctions_agree': 18, 'Crisis causes_Living beyond their means_agree': 11, 'Responsibility_Greek EMU entry_Red-green_agree': 13, 'Crisis causes_Greece_Lack of competitiveness_agree': 16, 'Crisis causes_Speculation_agree': 24, 'EU_Peace_agree': 19, "Aid_EU's future_agree": 24, 'Aid_Lawfulness_agree': 2, 'EMU_Sovereign insolvency procedure_agree': 11, 'EU_No bailout clause_agree': 29, 'Greece_Debt restructuring_agree': 17, 'Aid_Private sector involvement_agree': 29, 'Aid_Private creditors_agree': 4, 'Crisis causes_Greece_Mismanagement_agree': 20, 'Crisis causes_EU_SGP_agree': 4, 'Aid_Guaranty_agree': 39, 'Aid_Risk_agree': 23, 'Greece_Wage cuts_agree': 7, 'Crisis solution_Competitiveness_agree': 28, 'Greece_Political will_agree': 36, 'Aid_Wrong incentives_agre